In [1]:
import re
from urllib.request import urlopen
from urllib.request import Request
from bs4 import BeautifulSoup
import csv
import pandas as pd
import time
import datetime
import numpy as np
import os
import string
import gzip
import io

In [3]:
def format_filename(s):
#Take a string and return a valid filename constructed from the string. 
#Uses a whitelist approach: any characters not present in valid_chars are removed. 
    valid_chars = "-_.() %s%s" % (string.ascii_letters, string.digits)
    filename = ''.join(c for c in s if c in valid_chars)
    return filename

In [9]:
#can only access limited results
pagenumber=list(range(1,156))

In [5]:
template = 'http://searchen.chinadaily.com.cn/search?sortBy=-publishtime&view=allsitesppublished&classify=en&navigation=+%2Bsource%3A%22China+Daily%22&drillDown=&drillUp=&offset={}&query=democracy'
pagelinks = [template.format(x) for x in pagenumber]

In [6]:
for links,numbers in zip(pagelinks,pagenumber):
    time.sleep(1)
    filename = 'p{}.htm'.format(numbers)
    html = urlopen(links).read().decode('utf-8') 
    out_dir='chinadailyindex'
    os.makedirs(out_dir,exist_ok=True)
    with open(out_dir + "/" + filename, 'w',encoding='utf-8') as f:
        f.write(html)

In [40]:
title=[]
link=[]
date=[]
description=[]
for page in pagenumber:
    filename='p{}.htm'.format(page)
    html=open('chinadailyindex'+'/'+filename,'r',encoding='utf-8').read()
    soup = BeautifulSoup(html, "lxml")
    table = soup.find('ul',attrs={'class':'cs_sear_list'})
    for each in table.find_all('li'):
        try:
            tit=each.find('a').get_text()
        except:
            tit=None
        try:
            lin = each.find('a')['href'] #the link and description info can be directly scrapped from the main page
        except:
            lin=None        
        try:
            da=each.find('span',attrs={'class':'time'}).get_text()
            dat=re.search('(\d+)-(\d+)-(\d+)', da).group()
        except:
            dat=None
        try:
            des=each.find('p').get_text()
        except:
            des=None           
        
        title.append(tit) #append the list for each iteration
        link.append(lin)       
        date.append(dat)
        description.append(des)

In [41]:
#select only 2013-01-01 to 2015-12-31
title=title[45:-16]
link=link[45:-16]
date=date[45:-16]
description=description[45:-16]
df=pd.DataFrame({'title':title,'link':link,'date':date,'description':description}) 
df

,date,description,link,title
0,2015-12-31,...education institutions were more interested...,http://www.chinadaily.com.cn/hkedition/2015-12...,HK people must work together more
1,2015-12-31,...maintain world peace under Washington's lea...,http://www.chinadaily.com.cn/opinion/2015-12/3...,US will benefit by accepting China's rise
2,2015-12-15,...morality are not the same thing. When someo...,http://www.chinadaily.com.cn/hkedition/2015-12...,Growing activism shows that HK is becoming mor...
3,2015-12-15,The late Elsie Tu's concept of democracy is ar...,http://www.chinadaily.com.cn/hkedition/2015-12...,Elsie Tu's lifelong campaign for good governan...
4,2015-12-14,...told newly minted MPs from her opposition N...,http://www.chinadaily.com.cn/cndy/2015-12/14/c...,Suu Kyi's party literally cleans up
5,2015-12-09,...underprivileged and promoting pragmatic dem...,http://www.chinadaily.com.cn/hkedition/2015-12...,Tu tributes cross the political divide
6,2015-12-08,...our ethics to acknowledge these adverse res...,http://www.chinadaily.com.cn/cndy/2015-12/08/c...,Opposition wins majority in parliamentary elec...
7,2015-12-03,...by President U Thein Sein. Suu Kyi's Nation...,http://www.chinadaily.com.cn/cndy/2015-12/03/c...,"New leaders, president hold talks"
8,2015-12-01,...to meet Suu Kyi on Wednesday. Uncertainty s...,http://www.chinadaily.com.cn/cndy/2015-12/01/c...,IN BRIEF (Page 12)
9,2015-12-01,...trade treaty with Beijing after South Korea...,http://usa.chinadaily.com.cn/epaper/2015-12/01...,Free trade agreement with South Korea clears h...


In [42]:
df.to_csv('chinadaily.csv')

In [64]:
for linkk,datee,titlee,desc in zip(link,date,title,description): 
#the cndy collections need paid subscription to read, so I use description as text
    time.sleep(1)
    if re.search('chinadaily.com.cn/cndy',linkk) is None:
        try:    
            request = Request(linkk)
            request.add_header('Accept-encoding', 'gzip')
            response = urlopen(request)
            if response.info().get('Content-Encoding') == 'gzip':
                buf = io.BytesIO(response.read())
                f = gzip.GzipFile(fileobj=buf)
                html = f.read()
            else:
                html=response.read()
            soup = BeautifulSoup(html,'lxml')
            data=soup.find('div',attrs={'id':'Content'}).get_text()
        except:
            data=desc
    else:
        data=desc
    dt=' '.join([datee, titlee])
    name = '{}.txt'.format(dt)
    filename=format_filename(name)
    out_dir='chinadaily'
    os.makedirs(out_dir,exist_ok=True)
    with open(out_dir + "/" + filename, 'w',encoding='utf-8') as f:
        f.write(data)

In [ ]:
df=pd.read_csv('')

In [ ]:
import io

In [ ]:
response = urlopen('http://www.chinadaily.com.cn/hkedition/2016-03/12/content_23835005.htm')
buf = io.BytesIO(response.read())
f = gzip.GzipFile(fileobj=buf)
html = f.read()
soup = BeautifulSoup(html,'lxml')
data=soup.find('div',attrs={'id':'Content'}).get_text()
data

In [ ]:
html = try_request('http://searchen.chinadaily.com.cn/search?sortBy=-publishtime&view=allsitesppublished&classify=en&navigation=&drillDown=&drillUp=&offset=2&query=democracy').read().decode('utf-8') 
filename='x.txt'
soup = BeautifulSoup(html, "lxml")
table = soup.find('ul',attrs={'class':'cs_sear_list'}).get_text()
out_dir='chinadailyindex'
os.makedirs(out_dir,exist_ok=True)
with open(out_dir + "/" + filename, 'w',encoding='utf-8') as f:
    f.write(table)